In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Dependencies which require restarting 
!pip install trl
%pip install -U datasets
!pip install unbabel-comet
!pip install pyarrow==11.0.0

from huggingface_hub import notebook_login
notebook_login()

In [1]:
%env TOKENIZERS_PARALLELISM=false

env: TOKENIZERS_PARALLELISM=false


In [2]:
# Importing dependencies
import pandas as pd
import torch
import numpy as np
from transformers import NllbTokenizerFast, TrainingArguments, AutoModelForSeq2SeqLM
from trl import PPOTrainer, PPOConfig, create_reference_model
from datasets import Dataset
import warnings
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")
from collections import deque
from datasets import load_dataset
from trl import IterativeSFTTrainer
from comet import download_model, load_from_checkpoint
import statistics
import torch.nn.functional as F
import logging
loggers = [logging.getLogger(name) for name in logging.root.manager.loggerDict]
for logger in loggers:
    logger.setLevel(logging.WARNING)

# Setting device to GPU
device = "cuda:0" if torch.cuda.is_available() else "cpu"

#Loading reward generation model
model_path = download_model("Unbabel/wmt22-cometkiwi-da")
reward_gen_model = load_from_checkpoint(model_path)


german_pronouns = ['Er','Es','Sie','er','es','sie']

2024-08-12 19:07:30.598769: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-12 19:07:30.598869: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-12 19:07:30.734784: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

model.ckpt:   0%|          | 0.00/2.26G [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.07k [00:00<?, ?B/s]

LICENSE:   0%|          | 0.00/20.8k [00:00<?, ?B/s]

hparams.yaml:   0%|          | 0.00/710 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/513 [00:00<?, ?B/s]

In [13]:
# Reward functions
def gen_hf(data,token_length,pro_data):
    data_comet_inp = []
    for i in range(len(data)):
        data_comet = {}
        data_comet["mt"]=data[i]["mt"]
        data_comet["src"]=data[i]["src"]
        #data_comet["ref"]=data[i]["ref"]
        data_comet_inp.append(data_comet)
    output = reward_gen_model.predict(data_comet_inp,batch_size = len(data), gpus = 1,progress_bar=False)
    rewards = output.scores
    fin = []
    fin_alt = []
    '''
    for i in range(len(data)):
        print(f"For candidate translation {i}: ")
        print("Translation is: ", data[i]["mt"])
        print("Comet rewards: ",rewards[i])
        print("Pronoun rewards: ",pro_data[i])
        print("Final reward: ",rewards[i] + (pro_data[i]/token_length))
        fin.append(rewards[i] + (pro_data[i]/token_length))
        print("Final reward(alt): ",rewards[i] + (pro_data[i]/10))
        fin_alt.append(rewards[i] + (pro_data[i]/10))
        print("\n")
    '''
    #print("Comet rewards: ",rewards)
    pro_rew = pro_data
    #print("Pronoun rewards: ",pro_rew)
    final_reward = [0]*len(rewards)
    for i in range(len(final_reward)):
        final_reward[i] = rewards[i] + (pro_rew[i]/token_length)
        #final_reward[i] = rewards[i] 
    # /20 
    # /(No of tokens in the sentence)
    # Discard sentences with length <= 5 in training set 
    #print("\n")
    #print("Final rewards: ",final_reward)    
    high_ind = np.argmax(final_reward)
    y_high = data[high_ind]["mt"]
    return y_high, high_ind 

def calc_comet_score(data):
    output = reward_gen_model.predict(data,batch_size = len(data), gpus = 1,progress_bar=False)
    rewards = output.scores
    #print("COMET reward: ",rewards[0])
    return rewards[0]

def calc_val_comet_score(data):
    output = reward_gen_model.predict(data,batch_size = len(data), gpus = 1,progress_bar=False)
    rewards = output.scores
    return sum(rewards)/len(rewards)

def get_pronoun_reward(response,k,cts):
    pro_reward = 0
    german_pros = ['Er','Es','Sie','er','es','sie']
    fl = False
    for i in range(len(response.sequences[0])):
        token = tokenizer.decode(response.sequences[0][i])
        for i in range(int(len(german_pros)/2)):
            if (token == german_pros[i] or token == german_pros[i+3]):
                    probabilities_tensor = torch.nn.functional.softmax(response.scores[i-1], dim=-1)
                    max_prob_index = torch.argmax(probabilities_tensor)
                    max_prob_value = probabilities_tensor.flatten()[max_prob_index]

                    fl = True
                    if tgt_pronouns[k] == german_pros[i]:
                        pro_reward = max_prob_value.item()
                        cts[2]+=1
                    elif tgt_pronouns[k] == german_pros[i+3]:
                        pro_reward = max_prob_value.item()
                        cts[2]+=1
                    else:
                        pro_reward = -max_prob_value.item()
                        cts[0]+=1
    
    if fl==False:
        cts[1]+=1
    return pro_reward,cts


def calc_pronoun_score(responses):
    pro_reward = [0]*(len(responses))
    k = -84
    for i in range(len(responses)):
        for j in range(len(responses[i].sequences[0])):
            token = tokenizer.decode(responses[i].sequences[0][j])
            if token in german_pronouns:
                probabilities_tensor = torch.nn.functional.softmax(responses[i].scores[j-1], dim=-1)
                max_prob_index = torch.argmax(probabilities_tensor)
                max_prob_value = probabilities_tensor.flatten()[max_prob_index]
                if token == tgt_pronouns[k]:
                    pro_reward[i] = max_prob_value.item()
                else:
                    pro_reward[i] = -max_prob_value.item()
        k = k + 1
    return pro_reward

In [7]:
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/nllb-200-distilled-600M",token=True)
tokenizer = NllbTokenizerFast.from_pretrained("facebook/nllb-200-distilled-600M",token=True, src_lang = "eng_Latn", tgt_lang="deu_Latn",use_fast="False")
#tokenizer_rev = NllbTokenizerFast.from_pretrained("facebook/nllb-200-distilled-600M",token=True, src_lang = "deu_Latn", tgt_lang="eng_Latn",use_fast="False")
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Reading europarl dataset for ENGLISH ====> GERMAN
# Preprocess data

f = pd.read_csv("/kaggle/input/europarl-pronoun-data-it/filtered_europarl_pronoun_dataset_uniform_mult_pro.csv")

#f = open("/kaggle/input/europarl-nozip/europarl-v7.de-en.en")
#g = open("/kaggle/input/europarl-nozip/europarl-v7.de-en.de")


src_sents = f.iloc[:,0].tolist()
tgt_sents = f.iloc[:,1].tolist()
tgt_pronouns = f.iloc[:,2].tolist()

#src_sents = f.readlines()
#tgt_sents = g.readlines()

config.json:   0%|          | 0.00/846 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.55k [00:00<?, ?B/s]

In [11]:
# Loading transformer
VAL_SIZE = 84
train_src = src_sents[:-VAL_SIZE]
val_src = src_sents[-VAL_SIZE:]
train_tgt = tgt_sents[:-VAL_SIZE]
val_tgt = tgt_sents[-VAL_SIZE:]
valid_data = pd.DataFrame({'src_sents':train_src, 'tgt_sents':train_tgt})
BATCH_SIZE = 4
valid_data_hf = Dataset.from_pandas(valid_data)
def collator(data):
    return dict((key, [d[key] for d in data]) for key in data[0])

#Preprocess validation set
validation = pd.DataFrame({'src_sents':val_src, 'tgt_sents':val_tgt})


In [ ]:
import re 

# Training loop
# Loading model parameters and initialising trainer
# model_ref = create_reference_model(model)
device="cuda"
model = model.to(device)
model_ref = model.to(device)
dataloader = torch.utils.data.DataLoader(valid_data_hf, batch_size=BATCH_SIZE, shuffle=False, collate_fn = collator)
steps = 0
ite = 0
rewards = []
plot_s_reward = []
plot_p_reward = []
plot_fin_reward = []
plot_fin_alt_reward = []
avg_rew = deque(maxlen=15)
avg_fin_reward_dq =deque(maxlen=15)
avg_fin_alt_reward_dq =deque(maxlen=15)
avg_s_rew_dq = deque(maxlen=15)
avg_p_rew_dq = deque(maxlen=15)
avg_s_rew = []
avg_p_rew = []
avg_rew2 = []
avg_fin_reward = []
avg_fin_alt_reward = []
plot_pb_reward = []
plot_m_reward = []
plot_fin_batch_reward = []
plot_fin_alt_batch_reward = []
avg_pb_rew_dq = deque(maxlen=15)
avg_fin_batch_reward_dq = deque(maxlen=15)
avg_fin_alt_batch_reward_dq = deque(maxlen=15)
avg_m_rew_dq = deque(maxlen=15)
avg_pb_rew = []
avg_fin_batch_rew = []
avg_m_rew = []
training_args = TrainingArguments(
    per_device_train_batch_size=4, 
    gradient_accumulation_steps=16,
    output_dir="/kaggle/working/",
    learning_rate=6e-6,
    overwrite_output_dir=True,
    max_steps=1000,
    gradient_checkpointing=True,
    optim="adafactor",
    report_to="none",
    save_strategy="no",
    logging_strategy="no",
    logging_steps=None
)
val_scores = []
val_pronoun_scores = []
fin_scores = []
fin_scores_alt = []
no_of_positives = []
trainer = IterativeSFTTrainer(model=model,args=training_args,tokenizer=tokenizer)
import os
os.environ['WANDB_DISABLED'] = 'true' 
cts = [0,0,0]
for sample in dataloader:
    ite += 1
    sample_high = []
    sample_label = []
    print("###############################################")
    print("Batch no: ",ite)
    print("\n")
    query = tokenizer(sample["src_sents"],return_tensors="pt",padding=True,truncation=True).input_ids.to(device)
    batch_reward = []
    batch_fin_reward = []
    batch_fin_alt_reward = []
    batch_pronoun_reward = []
    input_query_list = []
    input_responses_list = []
    avg_reward = []
    #print("Source",sample["src_sents"])
    is_pro_correct = 0
    token_lengths = []
    for j in range(BATCH_SIZE):
        print("Sample no: ",j+1)
        rew = []
        unique_translations = []
        pronoun_reward = []
        it = 0
        
        '''match = re.search(r'</context>(.*)', sample["src_sents"][j])
        if match:
            relevant_text = match.group(1).strip()
        else:
            raise ValueError("The provided text does not contain '</context>'")
        
        token_length = len(tokenizer.tokenize(relevant_text))
        '''
        
        token_length = len(tokenizer.tokenize(sample["src_sents"][j]))
        #here
        '''
        print("TARGET SENTENCE: ",sample["tgt_sents"][j])
        print("PRONOUN IN TARGET SENTENCE: ",tgt_pronouns[j+(4*(ite-1))])
        print(f"Token length of the source sentence: {token_length}")
        print("\n")
        '''
        while len(unique_translations)<10:
            it = it + 1
            if it>=25 and len(unique_translations)<4:
                break
            if it>=50:
                break
            response = model.generate(
                    tokenizer(sample["src_sents"][j],return_tensors="pt",padding=True,truncation=True).input_ids.to(device),
                    forced_bos_token_id=tokenizer.lang_code_to_id["deu_Latn"],
                    do_sample = True,
                    num_beams= 1,
                    return_dict_in_generate=True, 
                    output_scores=True
                    )
            translation = tokenizer.decode(response.sequences[0], skip_special_tokens=True)
            pronoun_reward.append(get_pronoun_reward(response,j+(4*(ite-1)),[0,0,0])[0])
            # print(response.scores)
            flag = True
            for i in unique_translations:
                if translation == i:
                    #here
                    pronoun_reward = pronoun_reward[:-1]
                    flag = False
            if flag:
                unique_translations.append(translation)
        inp_ref = []
        inp_src = []
        for i in range(len(unique_translations)):
            inp_ref.append(sample["tgt_sents"][j])
            inp_src.append(sample["src_sents"][j])
        comet_input = []
        #print("Refs",sample["tgt_sents"][j])
        for i in range(len(unique_translations)):
            data = {}
            data["mt"]=unique_translations[i]
            data["src"]=sample["src_sents"][j]
            #here
            #data["pro"]=pronoun_reward[i]
            #data["ref"]=sample["tgt_sents"][j]
            comet_input.append(data)                                                                                                                                                                
        #print(comet_input)
        y_high, ind_best= gen_hf(comet_input,token_length,pronoun_reward)

        #Train Reward Plot
        model_best_response = model.generate(
                    tokenizer(sample["src_sents"][j],return_tensors="pt",padding=True,truncation=True).input_ids.to(device),
                    forced_bos_token_id=tokenizer.lang_code_to_id["deu_Latn"],
                    do_sample = True,
                    top_k=5,
                    return_dict_in_generate=True, 
                    output_scores=True
        )
        translation = tokenizer.decode(model_best_response.sequences[0],skip_special_tokens=True)
        #here
        print("\n Model best response: ",translation)
        pro_rew,cts = get_pronoun_reward(model_best_response,j+(4*(ite-1)),cts)
        print("Pronoun reward: ",pro_rew)
        '''
        if steps%25 == 0:
            print(f"For sent {j} in batch: ")
            print("Source sentence with context: ",sample["src_sents"][j])
            print("Source sentence without context: ",relevant_text)
            print("Best candidate translation: ",unique_translations[ind_best])
            print("Best translation for plot: ",translation)
            print("Ref (actual) translation: ",sample["tgt_sents"][j])
            print("Pronoun reward: ",pro_rew)
        '''
        comet_input = []
        data = {}
        data["mt"] = translation
        data["src"] = sample["src_sents"][j]
        #data["ref"] = sample["tgt_sents"][j]
        comet_input.append(data)
        highest = calc_comet_score(comet_input)
        plot_p_reward.append(pro_rew)
        plot_s_reward.append(highest)
        avg_p_rew_dq.append(pro_rew)
        avg_s_rew_dq.append(highest)
        avg_p_rew.append(np.mean(avg_p_rew_dq))
        avg_s_rew.append(np.mean(avg_s_rew_dq))
        batch_pronoun_reward.append(pro_rew)
        batch_reward.append(highest)
        sample_high.append(y_high)
        batch_fin_reward.append((pro_rew/token_length)+highest)
        batch_fin_alt_reward.append((pro_rew/10)+highest)
        token_lengths.append(token_length)
        print("\n")
    sample_src = sample['src_sents']
    mean_r = np.mean(batch_reward)
    mean_p = np.mean(batch_pronoun_reward)
    print("Batch mean train reward: ",mean_r)
    print("Batch mean pronoun train reward: ",mean_p)
    plot_m_reward.append(mean_r)
    plot_pb_reward.append(mean_p)
    avg_m_rew_dq.append(mean_r)
    avg_pb_rew_dq.append(mean_p)
    avg_m_rew.append(np.mean(avg_m_rew_dq))
    avg_pb_rew.append(np.mean(avg_pb_rew_dq))
    plot_fin_batch_reward.append(np.mean(batch_fin_reward))
    plot_fin_alt_batch_reward.append(np.mean(batch_fin_alt_reward))
    avg_fin_reward_dq.append(np.mean(batch_fin_reward))
    avg_fin_alt_reward_dq.append(np.mean(batch_fin_alt_reward))
    avg_fin_reward.append(np.mean(avg_fin_reward_dq))
    avg_fin_alt_reward.append(np.mean(avg_fin_alt_reward_dq))

    trainer.step(texts=sample_src,texts_labels=sample_high)
    steps+=1
    print("###############################################")
    print("\n")
    if steps==1 or steps%100==0:
        print("#########################################################")
        print("Performing Validation")
        print("#########################################################")
        src_sents = validation["src_sents"].tolist()
        tgt_sents = validation["tgt_sents"].tolist()
        translations = []
        best_responses = []
        src_wo_context = []
        print("Current no of wrong, no and right pronouns are: ",cts)
        for i in range(VAL_SIZE):
            '''
            match = re.search(r'</context>(.*)', src_sents[i])
            if match:
                relevant_text = match.group(1).strip()
            else:
                raise ValueError("The provided text does not contain '</context>'")
            src_wo_context.append(relevant_text)
            '''
            src_wo_context.append(src_sents[i])
            val_query = tokenizer(src_sents[i],return_tensors="pt",padding=True,truncation=True).input_ids.to(device)
            model_best_response = model.generate(
                    val_query,
                    forced_bos_token_id=tokenizer.lang_code_to_id["deu_Latn"],
                    do_sample = True,
                    top_k=5,
                    return_dict_in_generate=True, 
                    output_scores=True
                    )
            translations.append(tokenizer.decode(model_best_response.sequences[0], skip_special_tokens=True))
            best_responses.append(model_best_response)
        comet_input = []
        for i in range(len(validation)):
            data = {}
            data["mt"] = translations[i]
            data["src"] = src_wo_context[i]
            #data["ref"] = tgt_sents[i]
            comet_input.append(data)
        val_mean_score = calc_val_comet_score(comet_input)
        #here
        
        #pronoun_scores = calc_pronoun_score(best_responses)
        #val_mean_pronoun_score = sum(pronoun_scores)/len(pronoun_scores)
        #val_pronoun_scores.append(val_mean_pronoun_score)
        #pos = 0
        #for i in pronoun_scores:
        #    if i > 0:
        #        pos += 1
        #no_of_positives.append(pos)    
        #print("Current validation pronoun scores are: \n",val_pronoun_scores)
        #print("No of correct pronouns chosen are: \n",no_of_positives)
        
        val_scores.append(val_mean_score)
        print("Current validation Comet scores are: \n",val_scores)
        

    if (steps)%25==0:
        plt.plot(plot_s_reward,'b',label='Comet reward', alpha = 0.2)
        plt.plot(avg_s_rew,'r',label='Moving average (over last 15 samples)')
        plt.xlabel(f"No of samples")
        plt.ylabel("Samplewise Comet Training Reward")
        plt.legend()
        plt.savefig(f"/kaggle/working/Sample_comet_reward_{steps}_t5.png")
        plt.show()
        plt.clf()
        
        plt.plot(plot_p_reward,'b',label='Pronoun reward', alpha = 0.2)
        plt.plot(avg_p_rew,'r',label='Moving average (over last 15 samples)')
        plt.xlabel(f"No of samples")
        plt.ylabel("Samplewise Pronoun Training Reward")
        plt.legend()
        plt.savefig(f"/kaggle/working/Sample_pronoun_reward_{steps}_t5.png")
        plt.show()
        plt.clf()
        
        plt.plot(plot_pb_reward,'b',label='Pronoun reward', alpha = 0.2)
        plt.plot(avg_pb_rew,'r',label='Moving average (over last 15 samples)')
        plt.xlabel(f"No of batches")
        plt.ylabel("Batchwise Pronoun Training Reward")
        plt.legend()
        plt.savefig(f"/kaggle/working/Batch_pronoun_reward_{steps}_t5.png")
        plt.show()
        plt.clf()
        
        plt.plot(plot_m_reward,'b',label='Comet reward',alpha = 0.2)
        plt.plot(avg_m_rew,'r',label='Moving average (over last 15 samples)')
        plt.xlabel(f"No of batches")
        plt.ylabel("Batchwise Comet Training Reward")
        plt.legend()
        plt.savefig(f"/kaggle/working/Batch_comet_reward_{steps}_t5.png")
        plt.show()
        plt.clf()
        
        '''
        plt.plot(plot_fin_alt_batch_reward,'b',label='Pronoun reward (over tokens)', alpha = 0.2)
        plt.plot(avg_fin_alt_reward,'r',label='Moving average (over last 15 samples)')
        plt.xlabel(f"No of batches")
        plt.ylabel("Batchwise final training reward (alt)")
        plt.legend()
        plt.savefig(f"/kaggle/working/Batch_fin_alt_reward_{steps}_t5.png")
        plt.show()
        plt.clf()
        
        plt.plot(plot_fin_batch_reward,'b',label='Pronoun reward (over 10)', alpha = 0.2)
        plt.plot(avg_fin_reward,'r',label='Moving average (over last 15 samples)')
        plt.xlabel(f"No of batches")
        plt.ylabel("Batchwise final training reward")
        plt.legend()
        plt.savefig(f"/kaggle/working/Batch_fin_reward_{steps}_t5.png")
        plt.show()
        plt.clf()
        '''

        print("Current validation Comet scores are: \n",val_scores)
    if (steps)==750 or steps==1000:
        trainer.push_to_hub(f"satanicmangoes/europarl_pronoun_nllb_{steps}")
        break

max_steps is given, it will override any value given in num_train_epochs


###############################################
Batch no:  1


Sample no:  1

 Model best response:  Ich glaube auch, daß die Zeit gekommen ist, daß wir uns einer wesentlichen Überprüfung der Listen der GSP und GSP+ Begünstigten widmen.
Pronoun reward:  0


Sample no:  2

 Model best response:  Das ist doch nur die Sache, die wir in der Vergangenheit übernommen haben.
Pronoun reward:  0


Sample no:  3

 Model best response:  Ich danke also der Kommission für ihren Weißbuch und der Frau Rothe für ihren Bericht sowie allen anderen, die daran gearbeitet haben, ihn so bewundernswert und aussagekräftig zu machen.
Pronoun reward:  0


Sample no:  4

 Model best response:  Es ist daher dringend erforderlich, das zweite Binnenmarktpaket und das dritte Energiepaket zu implementieren. 
Pronoun reward:  1.0


Batch mean train reward:  0.7203928530216217
Batch mean pronoun train reward:  0.25
###############################################


#######################################################